# Matrix elements

In [45]:
import numpy as np

We begin by loading the two-body orbital matrix elements from `dat/coulomb.dat`. In this example the orbitals are equal for both spin-directions thus the spin-orbital matrix elements has twice the number of basis functions in each dimension. Labeling the orbital matrix elements by $(\tilde{p}\tilde{q}\lvert u \rvert \tilde{r}\tilde{s})$, we wish to set up the anti-symmetric matrix elements by

\begin{align}
    u^{pq}_{rs}
        &= \langle pq\lvert u\rvert rs\rangle
        - \langle pq\lvert u\rvert sr\rangle \\
        &= \delta_{\sigma_p\sigma_r}\delta_{\sigma_q\sigma_s}
        (\tilde{p}\tilde{q}\lvert u\rvert \tilde{r}\tilde{s})
        - \delta_{\sigma_p\sigma_s}\delta_{\sigma_q\sigma_r}
        (\tilde{p}\tilde{q}\lvert u\rvert \tilde{s}\tilde{r}),
\end{align}
where $p = (\tilde{p}, \sigma_p)$.

In [50]:
l = 12
orbital_integrals = np.zeros((l//2, l//2, l//2, l//2))

with open("dat/coulomb.dat", "r") as f:
    for row in f.read().split("\n"):
        row = row.split()

        if row:
            p, q, r, s, val = row
            orbital_integrals[int(p), int(q), int(r), int(s)] = float(val)

Having read in the orbital matrix elements, i.e., $(\tilde{p}\tilde{q}\lvert u\rvert \tilde{r}\tilde{s})$, we construct the anti-symmetric elements, $u^{pq}_{rs}$, using the formula shown above. We therefore introduce two $\delta$-functions. One for the spatial part and one for the spin part.

In [52]:
# Regular Kronecker-delta for indices
delta = lambda p, q: p == q
# Spin-delta, checking if both indices are odd or even
spin_delta = lambda p, q: not ((p & 0x1) ^ (q & 0x1))

In [54]:
u = np.zeros((l, l, l, l))

for p in range(l):
    for q in range(l):
        for r in range(l):
            for s in range(l):
                pqrs = spin_delta(p, r) * spin_delta(q, s) * orbital_integrals[p//2, q//2, r//2, s//2]
                pqsr = spin_delta(p, s) * spin_delta(q, r) * orbital_integrals[p//2, q//2, s//2, r//2]
                u[p, q, r, s] = pqrs - pqsr

Below we test if the anti-symmetric matrix elements are actually anti-symmetric, i.e.,

\begin{align}
    u_{rs}^{pq} = -u_{sr}^{pq} = -u_{rs}^{qp} = u_{sr}^{qp}.
\end{align}

In [40]:
tol = 1e-7

for p in range(l):
    for q in range(p + 1, l):
        for r in range(l):
            for s in range(r + 1, l):
                assert (abs(u[p, q, r, s] - u[q, p, s, r]) < tol)
                assert (abs(u[p, q, r, s] + u[q, p, r, s]) < tol)
                assert (abs(u[p, q, r, s] + u[p, q, s, r]) < tol)